In [1]:
from models.modules import MaxPooling, MaxPoolingX
import torch
import torch_geometric

from torch.nn import Linear
from torch.nn.functional import elu
from torch_geometric.nn.conv import SplineConv
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.transforms import Cartesian
from torch_geometric.data import Data, Batch
from torch_geometric.nn.pool import radius_graph, knn_graph

from models.modules import TransitionDown
from torch_geometric import seed_everything

seed_everything(0)

class GraphRes(torch.nn.Module):

    def __init__(self):
        super(GraphRes, self).__init__()
        dim = 3

        bias = False
        root_weight = False
        pooling_size=(16/346, 12/260)

        # Set dataset specific hyper-parameters.
        kernel_size = 2
        n = [1, 8, 16, 16, 16, 32, 32, 32, 32]
        pooling_outputs = 32
        #kernel_size = 8
        #n = [1, 16, 32, 32, 32, 128, 128, 128]
        #pooling_outputs = 128

        self.conv1 = SplineConv(n[0], n[1], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm1 = BatchNorm(in_channels=n[1])
        self.conv2 = SplineConv(n[1], n[2], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm2 = BatchNorm(in_channels=n[2])
        self.td2 = TransitionDown(n[2], n[2], 0.5)
        
        self.conv3 = SplineConv(n[2], n[3], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm3 = BatchNorm(in_channels=n[3])
        self.conv4 = SplineConv(n[3], n[4], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm4 = BatchNorm(in_channels=n[4])

        self.conv5 = SplineConv(n[4], n[5], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm5 = BatchNorm(in_channels=n[5])
        self.pool5 = MaxPooling(pooling_size, transform=Cartesian(norm=True, cat=False))

        self.conv6 = SplineConv(n[5], n[6], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm6 = BatchNorm(in_channels=n[6])
        self.conv7 = SplineConv(n[6], n[7], dim=dim, kernel_size=kernel_size, bias=bias, root_weight=root_weight)
        self.norm7 = BatchNorm(in_channels=n[7])

        self.pool7 = MaxPoolingX(0.25, size=16)
        self.fc = Linear(pooling_outputs * 16, out_features=2, bias=bias)

    def forward(self, data: torch_geometric.data.Batch) -> torch.Tensor:
        data.x = elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm1(data.x)
        data.x = elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm2(data.x)
        
        x, pos, batch = self.td2(data.x, data.pos, data.batch)
        edge_index = radius_graph(pos, r=0.05, max_num_neighbors=16)
        data = Batch(x=x, pos=pos, batch=batch, edge_index=edge_index, y=data.y)
        data = Cartesian()(data)

        x_sc = data.x.clone()
        data.x = elu(self.conv3(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm3(data.x)
        data.x = elu(self.conv4(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm4(data.x)
        data.x = data.x + x_sc

        data.x = elu(self.conv5(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm5(data.x)
        data = self.pool5(data.x, pos=data.pos, batch=data.batch, edge_index=data.edge_index, return_data_obj=True)

        x_sc = data.x.clone()
        data.x = elu(self.conv6(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm6(data.x)
        data.x = elu(self.conv7(data.x, data.edge_index, data.edge_attr))
        data.x = self.norm7(data.x)
        data.x = data.x + x_sc

        x = self.pool7(data.x, pos=data.pos[:, :2], batch=data.batch)
        x = x.view(-1, self.fc.in_features)
        return self.fc(x)

In [2]:
model = torch.load('../results/exp3/model').cpu()
model.load_state_dict(torch.load('../results/exp3/state_dict'))

<All keys matched successfully>

In [8]:
from imports.TactileDataset import TactileDataset
from torch_geometric.loader import DataLoader

train_1 = TactileDataset('../data/small/contact_extraction4/val/')
val_loader = DataLoader(train_1)

{'bag': '/home/hussain/me/projects/tactile/data/bags/small3', 'train_prop': 0.75, 'delta_t': 75000000.0, 'margin': -25000000.0, 'min_n_events': 750, 'n_init_events': -1, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 5, 'possible_angles': [0.15707963267948966], 'theta': [0, 0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966], 'augmentations': [], 'node_features': 'pol'}


In [9]:
import numpy as np
from tqdm.auto import tqdm

true = []
pred = []

loss_func = torch.nn.L1Loss()

for val in tqdm(val_loader):
    true.append(val.y)
    
    pred.append(model(val))

  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
out=np.degrees(([loss_func(pred[i].cpu(), true[i].cpu()).detach().item() for i in range(len(pred))]))
print(out)
np.mean(out)

[0.87534045 2.63362532 2.21480751 0.37399106 0.66148024 1.38350967
 1.6815327  2.48342562 2.28148788 1.36698606 1.48599602 1.85113046
 1.27258164 1.42165863 1.86424272]


1.5901197312158588

In [14]:
[train_1[i].x.shape for i in range(len(pred))]

[torch.Size([10675, 1]),
 torch.Size([18078, 1]),
 torch.Size([11711, 1]),
 torch.Size([18316, 1]),
 torch.Size([17069, 1]),
 torch.Size([16253, 1]),
 torch.Size([15656, 1]),
 torch.Size([12470, 1]),
 torch.Size([17199, 1]),
 torch.Size([11717, 1]),
 torch.Size([19018, 1]),
 torch.Size([15180, 1]),
 torch.Size([18391, 1]),
 torch.Size([14542, 1]),
 torch.Size([15270, 1])]

In [9]:
out=np.degrees(([loss_func(pred[i].cpu(), true[i].cpu()).detach().item() for i in range(len(pred))]))

In [11]:
np.mean(out)

2.2723606944138566

In [12]:
out

array([1.53873191, 0.90802349, 2.29150541, 2.5096354 , 2.63236259,
       1.19153547, 3.79068156, 3.36345515, 2.26212279, 5.14135403,
       2.54839779, 2.34648694, 3.6112081 , 2.4707712 , 2.520508  ,
       1.8478953 , 1.22903448, 1.29851266, 3.94667497, 4.65416774,
       1.06247328, 1.73233255, 3.89357751, 2.47369751, 0.86992021,
       2.17821819, 1.59466881, 0.84552358, 1.59278411, 2.26960719,
       2.24045829, 1.72670309, 1.46240551, 1.28046986, 4.47888118,
       3.25132799, 1.40947357, 2.39815861, 0.47641156, 0.75313764,
       0.62999173, 3.78218396, 0.82325866, 3.23094948, 3.69655217])

In [1]:
out

NameError: name 'out' is not defined